In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
import pandas as pd
import numpy as np
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(device)

cuda


In [ ]:
# define hyperparameters
input_size = 128
num_classes = 3
num_epochs = 10
batch_size = 64

In [ ]:
data_transforms = {
    # Random Rotation from -45 to 45 degree
    'train' :
    transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomRotation(degrees=(-100, -80)),
#        transforms.RandomHorizontalFlip(p=0.5), # 50% horizontal flip
#        transforms.RandomVerticalFlip(p=0.5), # 50% vertical flip
      #  transforms.RandomGrayscale(p=1), # 100% convert to gray
       transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.RandomRotation(degrees=(-93, -87)),
        transforms.ToTensor()
    ])
}

In [ ]:
# train dataset

data_dir = '/content/drive/MyDrive/PRS'
train_dir = data_dir + 'train'
test_dir = data_dir + 'test'

image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle = True) for x in ['train', 'test']}

In [ ]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 1500
     Root location: /content/drive/MyDrive/PRS/train
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
                RandomRotation(degrees=[-100.0, -80.0], interpolation=nearest, expand=False, fill=0)
                ToTensor()
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 150
     Root location: /content/drive/MyDrive/PRS/test
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            )}

In [ ]:
# // do dataset and convert to gray frames

# x_train = []
# y_train = []

# ROOT_DIR = "data_asl"
# train_dir = os.listdir(ROOT_DIR)[2]
# for letter in os.listdir(os.path.join(ROOT_DIR, train_dir)):
#     for img_path in os.listdir(os.path.join(ROOT_DIR, train_dir, letter)):
#         img = cv2.imread(os.path.join(ROOT_DIR, train_dir, letter, img_path))
#         seed = random.random()
#         if seed < 0.1:
#             x_train.append(img)
#             y_train.append(letter)

In [ ]:
# batches of data
train_loader = dataloaders["train"]

test_loader = dataloaders["test"]

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 3,
                out_channels = 64,
                kernel_size = 3,
                stride = 1,
                padding = 2,
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, 1, 2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.out = nn.Linear(12800, num_classes)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [ ]:
# define accuracy
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [ ]:
# Train a CNN model
# Instantializing
model = CNN()
# Loss Function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Start training
for epoch in range(num_epochs):
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} loss: {loss}")



        # right = accuracy(output, target)
        # train_rights.append(right)

# print("Final Accuracy:", sum(torch.argmax(model(x.to(device)), dim=1) == y.to(device) for x, y in test_loader) / len(test_loader))



        # if batch_idx % 10 == 0:
        #     model.eval()
        #     val_rights = []

        #     for (data, target) in test_loader:
        #         output = model(data)
        #         right = accuracy(output, target)
        #         val_rights.append(right)

            # Calculate Accuracy

            # train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            # val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            # print('Current Epoch: {} [{} / {} ({:.0f}%)]\t Loss: {:.6f}\t Train_set Accuracy: {:2f}%\t Test_set Accuracy: {:.2f}%'.format(
            #     epoch, batch_idx*batch_size, len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader),
            #     loss.data,
            #     100. * train_r[0].numpy() / train_r[1],
            #     100. * val_r[0].numpy() / val_r[1]))


Epoch 1 loss: 0.7136214375495911
Epoch 2 loss: 0.32064494490623474
Epoch 3 loss: 0.0819639340043068
Epoch 4 loss: 0.09355366975069046
Epoch 5 loss: 0.15723015367984772
Epoch 6 loss: 0.008798454888164997
Epoch 7 loss: 0.13351953029632568
Epoch 8 loss: 0.007511720526963472
Epoch 9 loss: 0.003151526441797614
Epoch 10 loss: 0.013999848626554012


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # images = images.to(device)
        # labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(150, 100 * correct / total))


Accuracy of the network on the 150 test images: 99.33333333333333 %


In [ ]:
torch.save(model, 'model.mdl')

In [ ]:
torch.save(model.state_dict(), 'weights_model.mdl')

In [ ]:
model2 = CNN()
model2.load_state_dict(torch.load('weights_model.mdl'))
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:

        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(150, 100 * correct / total))


Accuracy of the network on the 150 test images: 99.33333333333333 %


In [ ]:
model3 = torch.load('model.mdl')